In [ ]:
# install packages


# import packages



# format

# def stock_data(stock_name -> str):

# Return train_data, test_data 


# def  model_function(train_data -> np.array, num_of_predictions -> int):

# 	Return prediction 


# def evaluation(test_data -> np.array, prediction -> np.array):
	
# 	Return metrics
